In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pickle
import shutil
import os
import time
from collections import defaultdict
from PIL import Image
from matplotlib.pyplot import imsave

import facenet
import lfw
import foolbox

import mp

/home/ubuntu/anaconda3/envs/n2n/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/n2n/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/n2n/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu/anaconda3/envs/n2n/lib/python3.6/site-pack

In [2]:
import cv2

In [3]:
size = 100
faces1, faces2, labels = mp.load_testset(size)

## Getting the noise2clean TF graph

They use a very obscure and poorly designed nvidia research library called dnnlib. It doesn't have any documentation (I even answered a StackOverflow question about this after searching for a while https://stackoverflow.com/questions/55985317/is-there-any-documentation-for-dnnlib-additional-lib-in-projects-of-nvlabs)

In [4]:
import sys
sys.path.append('../../noise2noise/')

In [5]:
import dnnlib
import dnnlib.tflib
from dnnlib.tflib import tfutil

from collections import OrderedDict
from typing import List, Tuple, Union

In [6]:
# example of getting the source code of a function (used to get the entirity of dnnlib class function code)
import inspect
print(inspect.getsource(net.run))

# example of dynamically adding an instance method to a class
def run_test():
    class A():
        def __init__(self):
            self.a = 1
    def tmp(a_obj):
        print(a_obj.a)
    A.run = tmp
    a = A()
    a.run()
run_test()

In [7]:
def get_output_for(self, *in_expr, return_as_list: bool = False, **dynamic_kwargs):
    """Get TensorFlow expression(s) for the output(s) of this network, given the inputs."""
    assert len(in_expr) == self.num_inputs
    assert not all(expr is None for expr in in_expr)

    all_kwargs = dict(self.static_kwargs)
    all_kwargs.update(dynamic_kwargs)

    with tf.variable_scope(self.scope, reuse=tf.AUTO_REUSE):
        assert tf.get_variable_scope().name == self.scope
        valid_inputs = [expr for expr in in_expr if expr is not None]
        final_inputs = []
        for expr, name, shape in zip(in_expr, self.input_names, self.input_shapes):
            if expr is not None:
                expr = tf.identity(expr, name=name)
            else:
                expr = tf.zeros([tf.shape(valid_inputs[0])[0]] + shape[1:], name=name)
            final_inputs.append(expr)
        assert callable(self._build_func)
        out_expr = self._build_func(*final_inputs, **all_kwargs)

    assert tfutil.is_tf_expression(out_expr) or isinstance(out_expr, tuple)

    if return_as_list:
        out_expr = [out_expr] if tfutil.is_tf_expression(out_expr) else list(out_expr)

    return out_expr
dnnlib.tflib.network.Network.get_output_for = get_output_for

In [8]:
n2c_network_count = [-1]

In [9]:
def load_n2c_network():
    with open('network_final-gaussian-n2c.pickle', 'rb') as f:
        net = pickle.load(f)
        n2c_network_count[0] += 1
        return net

In [10]:
def make_graph(self,
        return_as_list: bool = False,
        print_progress: bool = False,
        minibatch_size: int = None,
        num_gpus: int = 1,
        assume_frozen: bool = False,
        out_mul: float = 1.0,
        out_add: float = 0.0,
        out_shrink: int = 1,
        out_dtype: np.dtype = None,
        **dynamic_kwargs) -> Union[np.ndarray, Tuple[np.ndarray, ...], List[np.ndarray]]:
    """Run this network for the given NumPy array(s), and return the output(s) as NumPy array(s).

    Args:
        return_as_list: True = return a list of NumPy arrays, False = return a single NumPy array, or a tuple if there are multiple outputs.
        print_progress: Print progress to the console? Useful for very large input arrays.
        minibatch_size: Maximum minibatch size to use, None = disable batching.
        num_gpus: Number of GPUs to use.
        assume_frozen: Improve multi-GPU perf by assuming that trainables are not going to change.
        out_mul: Multiplicative constant to apply to the output(s).
        out_add: Additive constant to apply to the output(s).
        out_shrink: Shrink the spatial dimensions of the output(s) by the given factor.
        out_dtype: Convert the output to the specified data type.
        dynamic_kwargs: Additional keyword arguments to pass into the network construction function.
    """
    
    graph = tf.Graph()
    
    with graph.as_default():
        key = str([list(sorted(dynamic_kwargs.items())), num_gpus, out_mul, out_add, out_shrink, out_dtype])

        # Build graph.
        if key not in self._run_cache:
            with tfutil.absolute_name_scope(self.scope + "/Run"), tf.control_dependencies(None):
                with tf.device("/cpu:0"):
                    in_expr = [tf.placeholder(tf.float32, name=name) for name in self.input_names]
                    in_split = list(zip(*[tf.split(x, num_gpus) for x in in_expr]))

                out_split = []
                for gpu in range(num_gpus):
                    with tf.device("/gpu:%d" % gpu):
                        net = self.clone() if assume_frozen else self
                        out_expr = net.get_output_for(*in_split[gpu], return_as_list=True, **dynamic_kwargs)

                        if out_mul != 1.0:
                            out_expr = [x * out_mul for x in out_expr]

                        if out_add != 0.0:
                            out_expr = [x + out_add for x in out_expr]

                        if out_shrink > 1:
                            ksize = [1, 1, out_shrink, out_shrink]
                            out_expr = [tf.nn.avg_pool(x, ksize=ksize, strides=ksize, padding="VALID", data_format="NCHW") for x in out_expr]

                        if out_dtype is not None:
                            if tf.as_dtype(out_dtype).is_integer:
                                out_expr = [tf.round(x) for x in out_expr]
                            out_expr = [tf.saturate_cast(x, out_dtype) for x in out_expr]

                        out_split.append(out_expr)

                with tf.device("/cpu:0"):
                    out_expr = [tf.concat(outputs, axis=0) for outputs in zip(*out_split)]
                    self._run_cache[key] = in_expr, out_expr

        in_expr, out_expr = self._run_cache[key]
                
        in_var_name = '{}'.format(net.input_names[0])
        out_var_name = '{}'.format(net.output_names[0])
#         return in_expr[0], out_expr[0], graph
        return in_var_name, out_var_name, graph
    
dnnlib.tflib.network.Network.make_graph = make_graph

In [120]:
# from tensorflow.python.client import device_lib

# device_lib.list_local_devices()

In [151]:
# img = faces1[0]

# w = img.shape[2]
# h = img.shape[1]
# pw, ph = (w+31)//32*32-w, (h+31)//32*32-h
# padded_img = img
# if pw!=0 or ph!=0:
#     padded_img  = np.pad(img, ((0,0),(0,ph),(0,pw)), 'reflect')

dnnlib.tflib.tfutil.init_tf()
    
def make_n2c_graph():
    net = load_n2c_network()
    return net.make_graph(width=256, height=256)

net.input_templates

# TODO: problem that with repeated runs of the graph creation
# the tensor gets new naming (since it's auto-naming)

# tf.reset_default_graph() -> last thing I tried, but doesn't seem to work directly ()

[<tf.Tensor 'autoencoder_20/x:0' shape=(?, 3, 256, 256) dtype=float32>]

In [152]:
print(n2c_network_count, net.output_templates)

[20] [<tf.Tensor 'autoencoder_20/dec_conv1/add:0' shape=(?, 3, 256, 256) dtype=float32>]


In [126]:
in_name, out_name, n2c_graph = make_n2c_graph()

g1_def = n2c_graph.as_graph_def()

In [127]:
# node_names = set(n.name for n in g1_def.node)
# sorted(node_names)
# [x in node_names for x in ['autoencoder_1/Run/x', 'autoencoder_1/Run/autoencoder_1/dec_conv1/add']]

In [128]:
faces1[0].shape
n2c_face = np.array([np.transpose(faces1[0], [2, 0, 1])])

In [132]:
n2c_face.shape

(1, 3, 160, 160)

In [130]:
ex_img_path = '../src/test/faces1-uniform/49.png'

In [140]:
# def preprocess_img_original(img):
#     img = img.astype(np.float32)
#     img = img.transpose([2, 0, 1]) / 255.0 - 0.5
#     w = img.shape[2]
#     h = img.shape[1]
#     pw, ph = (w+31)//32*32-w, (h+31)//32*32-h
#     padded_img = img
#     if pw!=0 or ph!=0:
#         padded_img  = np.pad(img, ((0,0),(0,ph),(0,pw)), 'reflect')
#     return np.expand_dims(padded_img, axis=0)

def preprocess_img(img):
    # im = Image.open(ex_img_path).convert('RGB')
    # img = cv2.imread(img_path)
    img = cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
    img = img.astype(np.float32)
    img = img.transpose([2, 0, 1]) / 255.0 - 0.5
    return np.expand_dims(img, axis=0)

In [158]:
img = preprocess_img(faces1[0])
img.shape

(1, 3, 256, 256)

In [118]:
def crop_np(img, x, y, w, h):
    return img[:, y:h, x:w]
def clip_to_uint8(arr):
    return np.clip((arr + 0.5) * 255.0 + 0.5, 0, 255).astype(np.uint8)

In [119]:
# import util
# network_snapshot = 'network_final-gaussian-n2c.pickle'
# tf_config = dnnlib.EasyDict()
# tf_config["graph_options.place_pruned_graph"] = True

# tfutil.init_tf(tf_config)
# net = util.load_snapshot(network_snapshot)
# im = Image.open(ex_img_path).convert('RGB')
# arr = np.array(im, dtype=np.float32)
# reshaped = arr.transpose([2, 0, 1]) / 255.0 - 0.5
# pred255 = util.infer_image(net, reshaped)
# t = pred255.transpose([1, 2, 0])  # [RGB, H, W] -> [H, W, RGB]
# # return PIL.Image.fromarray(t, 'RGB')

In [169]:
clean_img = net.run(img)[0]
x = clip_to_uint8(clean_img)

In [170]:
tf_config = dnnlib.EasyDict()
tf_config["graph_options.place_pruned_graph"] = True
tfutil.init_tf(tf_config)

in_name, out_name, n2c_graph = make_n2c_graph()
g1_def = n2c_graph.as_graph_def()

# https://stackoverflow.com/questions/33748552/tensorflow-how-to-replace-a-node-in-a-calculation-graph/33770771
with tf.Graph().as_default() as graph:
#     face_input = tf.placeholder(tf.float32, shape=(1, 160, 160, 3), name="input")
#     face_resized = tf.image.resize_images(face_input, [256, 256])
#     n2c_image_input = tf.transpose(face_resized, perm=[0, 2, 3, 1])
    
    n2c_image_input = tf.placeholder(tf.float32, shape=(1, 3, 256, 256), name="input")
    
    in_name = 'autoencoder_{}/Run/x:0'.format(n2c_network_count[0])
    out_name = 'autoencoder_{c}/Run/autoencoder_{c}/dec_conv1/add:0'.format(c=n2c_network_count[0])
    n2c_faces_orig, = tf.import_graph_def(
        g1_def,
        input_map={in_name: n2c_image_input}, return_elements=[out_name]
    )
    n2c_faces_transposed = tf.transpose(n2c_faces_orig, [0, 2, 3, 1])
    n2c_faces_resized = tf.image.resize_images(n2c_faces_transposed, [160, 160])

In [171]:
with tf.Session(graph=graph) as sess:
    res = sess.run(n2c_faces_resized, feed_dict={n2c_image_input: preprocess_img(faces1[0])})

FailedPreconditionError: Attempting to use uninitialized value import/autoencoder_25/enc_conv5/weight
	 [[Node: import/autoencoder_25/enc_conv5/weight/read = Identity[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](import/autoencoder_25/enc_conv5/weight)]]
	 [[Node: resize_images/ResizeBilinear/_3 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_323_resize_images/ResizeBilinear", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'import/autoencoder_25/enc_conv5/weight/read', defined at:
  File "/home/ubuntu/anaconda3/envs/n2n/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/n2n/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/envs/n2n/lib/python3.6/

In [163]:
res.shape

(1, 3, 256, 256)

## Inserting n2c graph into facenet, although this was dependant on the above working

In [25]:
class N2cInceptionResnetV1Model():
  model_path = "../models/20180402-114759/20180402-114759.pb"

  def __init__(self):
    """
    mean_filter_size -- size of a mean filter to be applied on every input image
    """
    # Load Facenet CNN
    facenet.load_model(self.model_path)
    # Save input and output tensors references
    graph = tf.get_default_graph()
    self.face_input = graph.get_tensor_by_name("input:0")
    self.embedding_output = graph.get_tensor_by_name("embeddings:0")
    
    self.graph = graph
    net = load_n2c_network()

    def make_n2c_graph():
        return net.make_graph(width=w+pw, height=h+ph)
    
    in_name, out_name, n2c_graph = make_n2c_graph()

    g1_def = n2c_graph.as_graph_def()
    g2_def = graph.as_graph_def()

    # https://stackoverflow.com/questions/33748552/tensorflow-how-to-replace-a-node-in-a-calculation-graph/33770771
    with tf.Graph().as_default() as g_combined:
        self.face_input = tf.placeholder(tf.float32, name="input")
        
        in_name = 'autoencoder_{}/Run/x:0'.format(n2c_network_count[0])
        out_name = 'autoencoder_{c}/Run/autoencoder_{c}/dec_conv1/add:0'.format(c=n2c_network_count[0])
        n2c_faces_orig, = tf.import_graph_def(
            g1_def,
            input_map={in_name: self.face_input}, return_elements=[out_name]
        )
        n2c_faces_transposed = tf.transpose(n2c_faces_orig, [0, 2, 3, 1])
        n2c_faces_resized = tf.image.resize_images(n2c_faces_transposed, [160, 160])
        self.embedding_output, = tf.import_graph_def(
            g2_def,
            input_map={"input:0": n2c_faces_resized, 'phase_train:0': False}, return_elements=["embeddings:0"]
        )

        # override the model's graph with our new combined graph
        self.graph = g_combined

  def convert_to_classifier(self):
    with self.graph.as_default():
        # Create victim_embedding placeholder
        self.victim_embedding_input = tf.placeholder(
            tf.float32,
            shape=(None, 512))

        # Squared Euclidean Distance between embeddings
        self.distance = tf.reduce_sum(
            tf.square(self.embedding_output - self.victim_embedding_input),
            axis=1)

        # Convert distance to a softmax vector
        # 0.99 out of 4 is the distance threshold for the Facenet CNN
        threshold = 0.99
        score = tf.where(
            self.distance > threshold,
            0.5 + ((self.distance - threshold) * 0.5) / (4.0 - threshold),
            0.5 * self.distance / threshold)
        reverse_score = 1.0 - score
        self.softmax_output = tf.transpose(tf.stack([reverse_score, score]))

        # Save softmax layer
        self.layer_names = []
        self.layers = []
        self.layers.append(self.softmax_output)
        self.layer_names.append('logits')

  def convert_to_targeted_classifier(self, victim):
    with self.graph.as_default():
        # Create victim_embedding placeholder
        self.victim_embedding_input = tf.constant(
            victim,
            shape=(1, 512))

        # Squared Euclidean Distance between embeddings
        self.distance = tf.reduce_sum(
            tf.square(self.embedding_output - self.victim_embedding_input),
            axis=1)

        # Convert distance to a softmax vector
        # 0.99 out of 4 is the distance threshold for the Facenet CNN
        threshold = 0.99
        score = tf.where(
            self.distance > threshold,
            0.5 + ((self.distance - threshold) * 0.5) / (4.0 - threshold),
            0.5 * self.distance / threshold)
        reverse_score = 1.0 - score

        self.softmax_output = tf.transpose(tf.stack([reverse_score, score]))

        # Foolbox expects logits
        self.logits = tf.log(self.softmax_output)

        # Save softmax layer
        self.layer_names = []
        self.layers = []
        self.layers.append(self.logits)
        self.layer_names.append('logits')

  def fprop(self, x, set_ref=False):
    return dict(zip(self.layer_names, self.layers))

In [61]:
facenet.load_model(N2cInceptionResnetV1Model.model_path)
# Save input and output tensors references
graph = tf.get_default_graph()
g2_def = graph.as_graph_def()

Model filename: ../models/20180402-114759/20180402-114759.pb


In [62]:
node_names = set([n.name for n in g2_def.node])

In [63]:
for n in sorted(node_names):
    print(n)

InceptionResnetV1/Block8/Branch_0/Conv2d_1x1/BatchNorm/Const
InceptionResnetV1/Block8/Branch_0/Conv2d_1x1/BatchNorm/Const_1
InceptionResnetV1/Block8/Branch_0/Conv2d_1x1/BatchNorm/Const_2
InceptionResnetV1/Block8/Branch_0/Conv2d_1x1/BatchNorm/Const_3
InceptionResnetV1/Block8/Branch_0/Conv2d_1x1/BatchNorm/beta
InceptionResnetV1/Block8/Branch_0/Conv2d_1x1/BatchNorm/beta/read
InceptionResnetV1/Block8/Branch_0/Conv2d_1x1/BatchNorm/beta/read_1
InceptionResnetV1/Block8/Branch_0/Conv2d_1x1/BatchNorm/beta/read_2
InceptionResnetV1/Block8/Branch_0/Conv2d_1x1/BatchNorm/beta/read_3
InceptionResnetV1/Block8/Branch_0/Conv2d_1x1/BatchNorm/beta_1
InceptionResnetV1/Block8/Branch_0/Conv2d_1x1/BatchNorm/beta_2
InceptionResnetV1/Block8/Branch_0/Conv2d_1x1/BatchNorm/beta_3
InceptionResnetV1/Block8/Branch_0/Conv2d_1x1/BatchNorm/cond/Const
InceptionResnetV1/Block8/Branch_0/Conv2d_1x1/BatchNorm/cond/Const_1
InceptionResnetV1/Block8/Branch_0/Conv2d_1x1/BatchNorm/cond/Const_1_1
InceptionResnetV1/Block8/Branch_0/

InceptionResnetV1/Conv2d_3b_1x1/BatchNorm/cond_1/switch_t_2
InceptionResnetV1/Conv2d_3b_1x1/BatchNorm/cond_1/switch_t_3
InceptionResnetV1/Conv2d_3b_1x1/BatchNorm/moving_mean
InceptionResnetV1/Conv2d_3b_1x1/BatchNorm/moving_mean/read
InceptionResnetV1/Conv2d_3b_1x1/BatchNorm/moving_mean/read_1
InceptionResnetV1/Conv2d_3b_1x1/BatchNorm/moving_mean/read_2
InceptionResnetV1/Conv2d_3b_1x1/BatchNorm/moving_mean/read_3
InceptionResnetV1/Conv2d_3b_1x1/BatchNorm/moving_mean_1
InceptionResnetV1/Conv2d_3b_1x1/BatchNorm/moving_mean_2
InceptionResnetV1/Conv2d_3b_1x1/BatchNorm/moving_mean_3
InceptionResnetV1/Conv2d_3b_1x1/BatchNorm/moving_variance
InceptionResnetV1/Conv2d_3b_1x1/BatchNorm/moving_variance/read
InceptionResnetV1/Conv2d_3b_1x1/BatchNorm/moving_variance/read_1
InceptionResnetV1/Conv2d_3b_1x1/BatchNorm/moving_variance/read_2
InceptionResnetV1/Conv2d_3b_1x1/BatchNorm/moving_variance/read_3
InceptionResnetV1/Conv2d_3b_1x1/BatchNorm/moving_variance_1
InceptionResnetV1/Conv2d_3b_1x1/BatchNor

InceptionResnetV1/Mixed_7a/Branch_1/Conv2d_0a_1x1/BatchNorm/cond_1/AssignMovingAvg_1/sub_1
InceptionResnetV1/Mixed_7a/Branch_1/Conv2d_0a_1x1/BatchNorm/cond_1/AssignMovingAvg_1/sub_2
InceptionResnetV1/Mixed_7a/Branch_1/Conv2d_0a_1x1/BatchNorm/cond_1/AssignMovingAvg_1/sub_3
InceptionResnetV1/Mixed_7a/Branch_1/Conv2d_0a_1x1/BatchNorm/cond_1/AssignMovingAvg_1_1
InceptionResnetV1/Mixed_7a/Branch_1/Conv2d_0a_1x1/BatchNorm/cond_1/AssignMovingAvg_1_2
InceptionResnetV1/Mixed_7a/Branch_1/Conv2d_0a_1x1/BatchNorm/cond_1/AssignMovingAvg_1_3
InceptionResnetV1/Mixed_7a/Branch_1/Conv2d_0a_1x1/BatchNorm/cond_1/AssignMovingAvg_2
InceptionResnetV1/Mixed_7a/Branch_1/Conv2d_0a_1x1/BatchNorm/cond_1/AssignMovingAvg_3
InceptionResnetV1/Mixed_7a/Branch_1/Conv2d_0a_1x1/BatchNorm/cond_1/AssignMovingAvg_4
InceptionResnetV1/Mixed_7a/Branch_1/Conv2d_0a_1x1/BatchNorm/cond_1/Identity
InceptionResnetV1/Mixed_7a/Branch_1/Conv2d_0a_1x1/BatchNorm/cond_1/Identity/Switch
InceptionResnetV1/Mixed_7a/Branch_1/Conv2d_0a_1x1/Ba

InceptionResnetV1/Repeat/block35_1/Branch_2/Conv2d_0b_3x3/BatchNorm/cond_1/Switch
InceptionResnetV1/Repeat/block35_1/Branch_2/Conv2d_0b_3x3/BatchNorm/cond_1/Switch_1
InceptionResnetV1/Repeat/block35_1/Branch_2/Conv2d_0b_3x3/BatchNorm/cond_1/Switch_1_1
InceptionResnetV1/Repeat/block35_1/Branch_2/Conv2d_0b_3x3/BatchNorm/cond_1/Switch_1_2
InceptionResnetV1/Repeat/block35_1/Branch_2/Conv2d_0b_3x3/BatchNorm/cond_1/Switch_1_3
InceptionResnetV1/Repeat/block35_1/Branch_2/Conv2d_0b_3x3/BatchNorm/cond_1/Switch_2
InceptionResnetV1/Repeat/block35_1/Branch_2/Conv2d_0b_3x3/BatchNorm/cond_1/Switch_3
InceptionResnetV1/Repeat/block35_1/Branch_2/Conv2d_0b_3x3/BatchNorm/cond_1/Switch_4
InceptionResnetV1/Repeat/block35_1/Branch_2/Conv2d_0b_3x3/BatchNorm/cond_1/pred_id
InceptionResnetV1/Repeat/block35_1/Branch_2/Conv2d_0b_3x3/BatchNorm/cond_1/pred_id_1
InceptionResnetV1/Repeat/block35_1/Branch_2/Conv2d_0b_3x3/BatchNorm/cond_1/pred_id_2
InceptionResnetV1/Repeat/block35_1/Branch_2/Conv2d_0b_3x3/BatchNorm/con

InceptionResnetV1/Repeat/block35_3/Branch_1/Conv2d_0a_1x1/BatchNorm/cond_1/AssignMovingAvg_1/sub_3
InceptionResnetV1/Repeat/block35_3/Branch_1/Conv2d_0a_1x1/BatchNorm/cond_1/AssignMovingAvg_1_1
InceptionResnetV1/Repeat/block35_3/Branch_1/Conv2d_0a_1x1/BatchNorm/cond_1/AssignMovingAvg_1_2
InceptionResnetV1/Repeat/block35_3/Branch_1/Conv2d_0a_1x1/BatchNorm/cond_1/AssignMovingAvg_1_3
InceptionResnetV1/Repeat/block35_3/Branch_1/Conv2d_0a_1x1/BatchNorm/cond_1/AssignMovingAvg_2
InceptionResnetV1/Repeat/block35_3/Branch_1/Conv2d_0a_1x1/BatchNorm/cond_1/AssignMovingAvg_3
InceptionResnetV1/Repeat/block35_3/Branch_1/Conv2d_0a_1x1/BatchNorm/cond_1/AssignMovingAvg_4
InceptionResnetV1/Repeat/block35_3/Branch_1/Conv2d_0a_1x1/BatchNorm/cond_1/Identity
InceptionResnetV1/Repeat/block35_3/Branch_1/Conv2d_0a_1x1/BatchNorm/cond_1/Identity/Switch
InceptionResnetV1/Repeat/block35_3/Branch_1/Conv2d_0a_1x1/BatchNorm/cond_1/Identity/Switch_1
InceptionResnetV1/Repeat/block35_3/Branch_1/Conv2d_0a_1x1/BatchNorm/c

...

InceptionResnetV1/Repeat_1/block17_7/Branch_0/Conv2d_1x1/BatchNorm/cond_1/AssignMovingAvg/sub/Switch_1
InceptionResnetV1/Repeat_1/block17_7/Branch_0/Conv2d_1x1/BatchNorm/cond_1/AssignMovingAvg/sub/Switch_1_1
InceptionResnetV1/Repeat_1/block17_7/Branch_0/Conv2d_1x1/BatchNorm/cond_1/AssignMovingAvg/sub/Switch_1_2
InceptionResnetV1/Repeat_1/block17_7/Branch_0/Conv2d_1x1/BatchNorm/cond_1/AssignMovingAvg/sub/Switch_1_3
InceptionResnetV1/Repeat_1/block17_7/Branch_0/Conv2d_1x1/BatchNorm/cond_1/AssignMovingAvg/sub/Switch_2
InceptionResnetV1/Repeat_1/block17_7/Branch_0/Conv2d_1x1/BatchNorm/cond_1/AssignMovingAvg/sub/Switch_3
InceptionResnetV1/Repeat_1/block17_7/Branch_0/Conv2d_1x1/BatchNorm/cond_1/AssignMovingAvg/sub/Switch_4
InceptionResnetV1/Repeat_1/block17_7/Branch_0/Conv2d_1x1/BatchNorm/cond_1/AssignMovingAvg/sub_1
InceptionResnetV1/Repeat_1/block17_7/Branch_0/Conv2d_1x1/BatchNorm/cond_1/AssignMovingAvg/sub_2
InceptionResnetV1/Repeat_1/block17_7/Branch_0/Conv2d_1x1/BatchNorm/cond_1/AssignM

InceptionResnetV1/Repeat_1/block17_9/Branch_1/Conv2d_0a_1x1/BatchNorm/cond/Merge_1_3
InceptionResnetV1/Repeat_1/block17_9/Branch_1/Conv2d_0a_1x1/BatchNorm/cond/Merge_2
InceptionResnetV1/Repeat_1/block17_9/Branch_1/Conv2d_0a_1x1/BatchNorm/cond/Merge_2_1
InceptionResnetV1/Repeat_1/block17_9/Branch_1/Conv2d_0a_1x1/BatchNorm/cond/Merge_2_2
InceptionResnetV1/Repeat_1/block17_9/Branch_1/Conv2d_0a_1x1/BatchNorm/cond/Merge_2_3
InceptionResnetV1/Repeat_1/block17_9/Branch_1/Conv2d_0a_1x1/BatchNorm/cond/Merge_3
InceptionResnetV1/Repeat_1/block17_9/Branch_1/Conv2d_0a_1x1/BatchNorm/cond/Merge_4
InceptionResnetV1/Repeat_1/block17_9/Branch_1/Conv2d_0a_1x1/BatchNorm/cond/Merge_5
InceptionResnetV1/Repeat_1/block17_9/Branch_1/Conv2d_0a_1x1/BatchNorm/cond/Switch
InceptionResnetV1/Repeat_1/block17_9/Branch_1/Conv2d_0a_1x1/BatchNorm/cond/Switch_1
InceptionResnetV1/Repeat_1/block17_9/Branch_1/Conv2d_0a_1x1/BatchNorm/cond/Switch_2
InceptionResnetV1/Repeat_1/block17_9/Branch_1/Conv2d_0a_1x1/BatchNorm/cond/Swi

InceptionResnetV1/Repeat_2/block8_2/Branch_1/Conv2d_0b_1x3/BatchNorm/cond/pred_id
InceptionResnetV1/Repeat_2/block8_2/Branch_1/Conv2d_0b_1x3/BatchNorm/cond/pred_id_1
InceptionResnetV1/Repeat_2/block8_2/Branch_1/Conv2d_0b_1x3/BatchNorm/cond/pred_id_2
InceptionResnetV1/Repeat_2/block8_2/Branch_1/Conv2d_0b_1x3/BatchNorm/cond/pred_id_3
InceptionResnetV1/Repeat_2/block8_2/Branch_1/Conv2d_0b_1x3/BatchNorm/cond/switch_t
InceptionResnetV1/Repeat_2/block8_2/Branch_1/Conv2d_0b_1x3/BatchNorm/cond/switch_t_1
InceptionResnetV1/Repeat_2/block8_2/Branch_1/Conv2d_0b_1x3/BatchNorm/cond/switch_t_2
InceptionResnetV1/Repeat_2/block8_2/Branch_1/Conv2d_0b_1x3/BatchNorm/cond/switch_t_3
InceptionResnetV1/Repeat_2/block8_2/Branch_1/Conv2d_0b_1x3/BatchNorm/cond_1/AssignMovingAvg
InceptionResnetV1/Repeat_2/block8_2/Branch_1/Conv2d_0b_1x3/BatchNorm/cond_1/AssignMovingAvg/Switch
InceptionResnetV1/Repeat_2/block8_2/Branch_1/Conv2d_0b_1x3/BatchNorm/cond_1/AssignMovingAvg/Switch_1
InceptionResnetV1/Repeat_2/block8_2/

InceptionResnetV1/Repeat_2/block8_4/Branch_1/Conv2d_0c_3x1/BatchNorm/cond/FusedBatchNorm_1/Switch_1_1
InceptionResnetV1/Repeat_2/block8_4/Branch_1/Conv2d_0c_3x1/BatchNorm/cond/FusedBatchNorm_1/Switch_1_2
InceptionResnetV1/Repeat_2/block8_4/Branch_1/Conv2d_0c_3x1/BatchNorm/cond/FusedBatchNorm_1/Switch_1_3
InceptionResnetV1/Repeat_2/block8_4/Branch_1/Conv2d_0c_3x1/BatchNorm/cond/FusedBatchNorm_1/Switch_2
InceptionResnetV1/Repeat_2/block8_4/Branch_1/Conv2d_0c_3x1/BatchNorm/cond/FusedBatchNorm_1/Switch_2_1
InceptionResnetV1/Repeat_2/block8_4/Branch_1/Conv2d_0c_3x1/BatchNorm/cond/FusedBatchNorm_1/Switch_2_2
InceptionResnetV1/Repeat_2/block8_4/Branch_1/Conv2d_0c_3x1/BatchNorm/cond/FusedBatchNorm_1/Switch_2_3
InceptionResnetV1/Repeat_2/block8_4/Branch_1/Conv2d_0c_3x1/BatchNorm/cond/FusedBatchNorm_1/Switch_3
InceptionResnetV1/Repeat_2/block8_4/Branch_1/Conv2d_0c_3x1/BatchNorm/cond/FusedBatchNorm_1/Switch_3_1
InceptionResnetV1/Repeat_2/block8_4/Branch_1/Conv2d_0c_3x1/BatchNorm/cond/FusedBatchNo

autoencoder_1/enc_conv1/bias/Assign
autoencoder_1/enc_conv1/bias/Initializer/zeros
autoencoder_1/enc_conv1/bias/Initializer/zeros/Const
autoencoder_1/enc_conv1/bias/Initializer/zeros/shape_as_tensor
autoencoder_1/enc_conv1/bias/new_value
autoencoder_1/enc_conv1/bias/read
autoencoder_1/enc_conv1/bias/setter
autoencoder_1/enc_conv1/weight
autoencoder_1/enc_conv1/weight/Assign
autoencoder_1/enc_conv1/weight/Initializer/random_normal
autoencoder_1/enc_conv1/weight/Initializer/random_normal/RandomStandardNormal
autoencoder_1/enc_conv1/weight/Initializer/random_normal/mean
autoencoder_1/enc_conv1/weight/Initializer/random_normal/mul
autoencoder_1/enc_conv1/weight/Initializer/random_normal/shape
autoencoder_1/enc_conv1/weight/Initializer/random_normal/stddev
autoencoder_1/enc_conv1/weight/new_value
autoencoder_1/enc_conv1/weight/read
autoencoder_1/enc_conv1/weight/setter
autoencoder_1/enc_conv2/Conv2D
autoencoder_1/enc_conv2/LeakyRelu/Maximum
autoencoder_1/enc_conv2/LeakyRelu/alpha
autoencode

In [26]:
model = N2cInceptionResnetV1Model()

Model filename: ../models/20180402-114759/20180402-114759.pb


## 3. An example attack on our merged model

In [27]:
# missing GPU bug https://stackoverflow.com/questions/38559755/how-to-get-current-available-gpus-in-tensorflow
# 

In [28]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [29]:
tf.__version__

'1.7.0'

In [30]:
get_available_gpus()

['/device:GPU:0']

In [31]:
# https://stackoverflow.com/questions/42326748/tensorflow-on-gpu-no-known-devices-despite-cudas-devicequery-returning-a-pas
# RIP! needed tensorflow-gpu...

In [33]:
tf.Graph().as_default()
# Convert to classifier
with model.graph.as_default():
    model.convert_to_classifier()

    # Load pairs of faces and their labels in one-hot encoding
    size = 100
    faces1, faces2, labels = mp.load_testset(size)
    binary_labels = [0 if labels[i][0] == 1 else 1 for i in range(size)]

    # Create victims' embeddings using Facenet itself
    graph = tf.get_default_graph()

    with tf.Session() as sess:
        feed_dict = {model.face_input: faces2}
        victims_embeddings = sess.run(model.embedding_output, feed_dict=feed_dict)
        feed_dict = {model.face_input: faces1}
        non_victims_embeddings = sess.run(model.embedding_output, feed_dict=feed_dict)

FailedPreconditionError: Attempting to use uninitialized value import/autoencoder_1/enc_conv5/weight
	 [[Node: import/autoencoder_1/enc_conv5/weight/read = Identity[T=DT_FLOAT, _class=["loc:@import/autoencoder_1/enc_conv5/weight"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](import/autoencoder_1/enc_conv5/weight)]]
	 [[Node: import_1/embeddings/_9 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_6778_import_1/embeddings", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'import/autoencoder_1/enc_conv5/weight/read', defined at:
  File "/home/ubuntu/anaconda3/envs/n2n_facenet/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/n2n_facenet/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  Fi